#  01 - Exploración Inicial Base de Pings y Shapefiles de OSM
Este notebook carga los datos de pings geolocalizados, hace una deduplicación y un analisis estructurado de datos sobre estos así como sobre los archivos shapefiles de OpenStreetMap correspondientes a El Salvador


## Archivo de Pings Geolocalizados
Correspondientes a Diciembre 2024 en El Salvador

In [0]:
%sql
select * from sv_12_2023 limit 40

Descriptivos Agrupados

In [0]:
%sql
select count(1),count(distinct device_id), count(distinct date), max(latitude),min(latitude),max(longitude),min(longitude) from sv_12_2023

Encontrando Valores Duplicados

In [0]:
%sql
select timestamp, device_id,latitude,longitude,count(1)
from sv_12_2023
group by 1,2,3,4
having count(1)>1

Eliminación de Duplicados y creación de variables de interés sobre la fecha

In [0]:
%sql
CREATE TABLE sv_12_2023_deduped AS
with raw as (
SELECT
  distinct  from_utc_timestamp(from_unixtime(timestamp), "America/El_Salvador") datetime,
  timestamp,
  device_id,
  latitude,
  longitude
FROM
  sv_12_2023
)
select datetime,
       hour(datetime) hora,
       date(datetime) fecha,
       CASE 
        WHEN dayofweek(datetime) IN (1, 7) THEN 1
        ELSE 0
       END AS is_weekend,
       CASE dayofweek(datetime)
        WHEN 1 THEN 'Domingo'
        WHEN 2 THEN 'Lunes'
        WHEN 3 THEN 'Martes'
        WHEN 4 THEN 'Miércoles'
        WHEN 5 THEN 'Jueves'
        WHEN 6 THEN 'Viernes'
        WHEN 7 THEN 'Sábado'
      END AS nombre_dia,
       timestamp,
       device_id,
       latitude,
       longitude
from raw


Cantidad de registros después de quitar duplicados

In [0]:
%sql
select count(1) from sv_12_2023_deduped

Distribución de pings y dispositivos únicos por día de semana

In [0]:
%sql
select concat(dayofweek(datetime),'.',nombre_dia) dia,count(1) registros, count(distinct device_id) dispositivos_unicos from sv_12_2023_deduped group by 1

Databricks visualization. Run in Databricks to view.

Histograma de pings por día por dispositivo

In [0]:
%sql
select concat(dayofweek(datetime),'.',nombre_dia) dia,fecha, device_id,count(1) pings_por_dia
from sv_12_2023_deduped group by 1,2,3

Databricks visualization. Run in Databricks to view.

BoxPlot de pings por día por dispositivo

In [0]:
%sql
select concat(dayofweek(datetime),'.',nombre_dia) dia,fecha, device_id,count(1) pings_por_dia
from sv_12_2023_deduped group by 1,2,3

Databricks visualization. Run in Databricks to view.

Tiempo promedio entre pings por dispositivo por dia

In [0]:
%sql
with datos as (
select fecha,
       device_id,
       timestamp,
       lag(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) last_timestamp,
       lead(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) next_timestamp
from sv_12_2023_deduped
)
, datos2 as (
select fecha,
       device_id,
       case when avg(timestamp-last_timestamp)<30 then 'C. < 30 segundos'
            when avg(timestamp-last_timestamp)<60 then 'D. < 1 minuto'
            when (avg(timestamp-last_timestamp)/60)<5 then 'E. < 5 minutos'
            when (avg(timestamp-last_timestamp)/60)<15 then 'F. < 15 minutos'
            when (avg(timestamp-last_timestamp)/60)<30 then 'G. < 30 minutos'
            when (avg(timestamp-last_timestamp)/60)<60 then 'H. < 1 hora'
            when (avg(timestamp-last_timestamp)/60/60)<5 then 'I. < 5 horas'
            when (avg(timestamp-last_timestamp)/60/60)<15 then 'J. < 15 horas'
          else 'K. >= 15 horas'
       end as categorias_tiempo_entre_pings
from datos
where last_timestamp is not null and next_timestamp is not null
group by 1,2
union all
select fecha,
       device_id,
       'A. unico ping' categorias_tiempo_entre_pings
from datos
where last_timestamp is null and next_timestamp is null
union all
select fecha,
       device_id,
       case when avg(next_timestamp-timestamp)<30 then 'C. < 30 segundos'
            when avg(next_timestamp-timestamp)<60 then 'D. < 1 minuto'
            when (avg(next_timestamp-timestamp)/60)<5 then 'E. < 5 minutos'
            when (avg(next_timestamp-timestamp)/60)<15 then 'F. < 15 minutos'
            when (avg(next_timestamp-timestamp)/60)<30 then 'G. < 30 minutos'
            when (avg(next_timestamp-timestamp)/60)<60 then 'H. < 1 hora'
            when (avg(next_timestamp-timestamp)/60/60)<5 then 'I. < 5 horas'
            when (avg(next_timestamp-timestamp)/60/60)<15 then 'J. < 15 horas'
          else 'K. >= 15 horas'
       end as categorias_tiempo_entre_pings
from datos
where last_timestamp is null and next_timestamp is not null
group by 1,2
)
select *
from datos2



Databricks visualization. Run in Databricks to view.

Analizando tiempo entre pings y distancia promedio entre pings

In [0]:

%sql
with datos as (
select fecha,
       device_id,
       timestamp,
       lag(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) last_timestamp,
       lead(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) next_timestamp,
        6371000 * 2 * ASIN(
        SQRT(
        POWER(SIN(RADIANS(latitude - lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2) +
        COS(RADIANS(latitude)) *
        COS(RADIANS(lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) )) *
        POWER(SIN(RADIANS(longitude - lag(longitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2)
        )
      ) AS distancia_metros,
       6371000 * 2 * ASIN(
        SQRT(
        POWER(SIN(RADIANS(latitude - lead(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2) +
        COS(RADIANS(latitude)) *
        COS(RADIANS(lead(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) )) *
        POWER(SIN(RADIANS(longitude - lead(longitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2)
        )
      ) AS distancia_metros_2
from sv_12_2023_deduped
)
, datos2 as (
select fecha,
       device_id,
       case when (timestamp-last_timestamp)<30 then 'C. < 30 segundos'
            when (timestamp-last_timestamp)<60 then 'D. < 1 minuto'
            when ((timestamp-last_timestamp)/60)<5 then 'E. < 5 minutos'
            when ((timestamp-last_timestamp)/60)<15 then 'F. < 15 minutos'
            when ((timestamp-last_timestamp)/60)<30 then 'G. < 30 minutos'
            when ((timestamp-last_timestamp)/60)<60 then 'H. < 1 hora'
            when ((timestamp-last_timestamp)/60/60)<5 then 'I. < 5 horas'
            when ((timestamp-last_timestamp)/60/60)<15 then 'J. < 15 horas'
          else 'K. >= 15 horas'
       end as categorias_tiempo_entre_pings,
       distancia_metros
from datos
where last_timestamp is not null and next_timestamp is not null
union all
select fecha,
       device_id,
       'A. unico ping' categorias_tiempo_entre_pings,
       0 distancia_metros
from datos
where last_timestamp is null and next_timestamp is null
union all
select fecha,
       device_id,
       case when (next_timestamp-timestamp)<30 then 'C. < 30 segundos'
            when (next_timestamp-timestamp)<60 then 'D. < 1 minuto'
            when ((next_timestamp-timestamp)/60)<5 then 'E. < 5 minutos'
            when ((next_timestamp-timestamp)/60)<15 then 'F. < 15 minutos'
            when ((next_timestamp-timestamp)/60)<30 then 'G. < 30 minutos'
            when ((next_timestamp-timestamp)/60)<60 then 'H. < 1 hora'
            when ((next_timestamp-timestamp)/60/60)<5 then 'I. < 5 horas'
            when ((next_timestamp-timestamp)/60/60)<15 then 'J. < 15 horas'
          else 'K. >= 15 horas'
       end as categorias_tiempo_entre_pings,
       distancia_metros_2 distancia_metros
from datos
where last_timestamp is null and next_timestamp is not null
)
select categorias_tiempo_entre_pings,count(1),median(distancia_metros)
from datos2
group by 1

Databricks visualization. Run in Databricks to view.

Acotamiento por categorias de los pings con tiempos menores a 30 segundos

In [0]:
%sql
with datos as (
select fecha,
       device_id,
       timestamp,
       lag(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) last_timestamp,
       lead(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) next_timestamp,
        6371000 * 2 * ASIN(
        SQRT(
        POWER(SIN(RADIANS(latitude - lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2) +
        COS(RADIANS(latitude)) *
        COS(RADIANS(lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) )) *
        POWER(SIN(RADIANS(longitude - lag(longitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2)
        )
      ) AS distancia_metros,
        6371000 * 2 * ASIN(
        SQRT(
        POWER(SIN(RADIANS(latitude - lead(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2) +
        COS(RADIANS(latitude)) *
        COS(RADIANS(lead(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) )) *
        POWER(SIN(RADIANS(longitude - lead(longitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2)
        )
      ) AS distancia_metros_2
from sv_12_2023_deduped
)
, datos2 as (
select fecha,
       device_id,
       case when (timestamp-last_timestamp)<5 then 'A. < 5 segundos'
            when (timestamp-last_timestamp)<10 then 'B. < 10 segundos'
            when (timestamp-last_timestamp)<15 then 'C. < 15 segundos'
            when (timestamp-last_timestamp)<20 then 'D. < 20 segundos'
            when (timestamp-last_timestamp)<25 then 'E. < 25 segundos'
          else 'F. < 30 segundos'
       end as categorias_tiempo_entre_pings,
       distancia_metros
from datos
where last_timestamp is not null and next_timestamp is not null
  and (timestamp-last_timestamp)<30 
union all
select fecha,
       device_id,
       case when (next_timestamp-timestamp)<5 then 'A. < 5 segundos'
            when (next_timestamp-timestamp)<10 then 'B. < 10 segundos'
            when (next_timestamp-timestamp)<15 then 'C. < 15 segundos'
            when (next_timestamp-timestamp)<20 then 'D. < 20 segundos'
            when (next_timestamp-timestamp)<25 then 'E. < 25 segundos'
          else 'F. < 30 segundos'
       end as categorias_tiempo_entre_pings,
       distancia_metros_2 distancia_metros
from datos
where last_timestamp is null and next_timestamp is not null
  and (next_timestamp-timestamp)<30
)
select categorias_tiempo_entre_pings,count(1),median(distancia_metros)
from datos2
group by 1

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
with datos as (
select fecha,
       device_id,
       timestamp,
       lag(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) last_timestamp,
       lead(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) next_timestamp,
       lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) last_latitude,
       lag(longitude,1,null) over (partition by device_id,fecha order by timestamp asc) last_longitude,
       latitude,
       longitude,
        6371000 * 2 * ASIN(
        SQRT(
        POWER(SIN(RADIANS(latitude - lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2) +
        COS(RADIANS(latitude)) *
        COS(RADIANS(lag(latitude,1,null) over (partition by device_id,fecha order by timestamp asc) )) *
        POWER(SIN(RADIANS(longitude - lag(longitude,1,null) over (partition by device_id,fecha order by timestamp asc) ) / 2), 2)
        )
      ) AS distancia_metros
from sv_12_2023_deduped
)
, datos2 as (
select fecha,
       device_id,
       case when (timestamp-last_timestamp)<30 then 'C. < 30 segundos'
            when (timestamp-last_timestamp)<60 then 'D. < 1 minuto'
            when ((timestamp-last_timestamp)/60)<5 then 'E. < 5 minutos'
            when ((timestamp-last_timestamp)/60)<15 then 'F. < 15 minutos'
            when ((timestamp-last_timestamp)/60)<30 then 'G. < 30 minutos'
            when ((timestamp-last_timestamp)/60)<60 then 'H. < 1 hora'
            when ((timestamp-last_timestamp)/60/60)<5 then 'I. < 5 horas'
            when ((timestamp-last_timestamp)/60/60)<15 then 'J. < 15 horas'
          else 'K. >= 15 horas'
       end as categorias_tiempo_entre_pings,
       distancia_metros
from datos
where last_timestamp is not null and next_timestamp is not null
  and (timestamp-last_timestamp)>=20 
  and ((timestamp-last_timestamp)/60/60)<5
union all
select fecha,
       device_id,
       case when (next_timestamp-timestamp)<30 then 'C. < 30 segundos'
            when (next_timestamp-timestamp)<60 then 'D. < 1 minuto'
            when ((next_timestamp-timestamp)/60)<5 then 'E. < 5 minutos'
            when ((next_timestamp-timestamp)/60)<15 then 'F. < 15 minutos'
            when ((next_timestamp-timestamp)/60)<30 then 'G. < 30 minutos'
            when ((next_timestamp-timestamp)/60)<60 then 'H. < 1 hora'
            when ((next_timestamp-timestamp)/60/60)<5 then 'I. < 5 horas'
            when ((next_timestamp-timestamp)/60/60)<15 then 'J. < 15 horas'
          else 'K. >= 15 horas'
       end as categorias_tiempo_entre_pings,
       distancia_metros
from datos
where last_timestamp is null and next_timestamp is not null
  and (next_timestamp-timestamp)>=20 
  and ((next_timestamp-timestamp)/60/60)<5
)
select categorias_tiempo_entre_pings,count(1),median(distancia_metros)
from datos2
group by 1

Databricks visualization. Run in Databricks to view.

Generando base final para ser utilizada en entrenamiento

In [0]:
%sql
CREATE TABLE sv_12_2023_filtered AS
with datos as (
select a.*,
       lag(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) last_timestamp,
       lead(timestamp,1,null) over (partition by device_id,fecha order by timestamp asc) next_timestamp
from sv_12_2023_deduped a
)
, datos2 as (
select *
from datos
where last_timestamp is not null and next_timestamp is not null
  and (timestamp-last_timestamp)>=20 
  and ((timestamp-last_timestamp)/60/60)<5
union all
select *
from datos
where last_timestamp is null and next_timestamp is not null
  and (next_timestamp-timestamp)>=20 
  and ((next_timestamp-timestamp)/60/60)<5
)
select *
from datos2

In [0]:
%sql
select * from sv_12_2023_filtered limit 40

## Archivo de Shapefiles de OpenStreetMaps

In [0]:
!pip install geopandas

In [0]:
import os
import geopandas as gpd
import pandas as pd
from shapely import wkt, Polygon

In [0]:
shapes_dir = "../Proyecto de Grado/shapes/"
gdf_list = []

for filename in os.listdir(shapes_dir):
    if filename.endswith(".geojson"):
        file_path = os.path.join(shapes_dir, filename)
        gdf = gpd.read_file(file_path)
        gdf['source_file'] = os.path.splitext(filename)[0]
        gdf_selected = gdf[
            ['source_file', 'osm_id', 'code', 'fclass', 'name', 'geometry']
        ]
        gdf_list.append(gdf_selected)

gdf_all = gpd.GeoDataFrame(
    pd.concat(gdf_list, ignore_index=True),
    crs=gdf_list[0].crs if gdf_list else None
)

In [0]:
gdf_all.display()

In [0]:
# Convert GeoDataFrame to Pandas DataFrame (geometry as WKT)
gdf_all['geometry_wkt'] = gdf_all['geometry'].apply(lambda geom: geom.wkt if geom is not None else None)
pdf = gdf_all.drop(columns='geometry')

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(pdf)

# Register as a temporary view or save as a table
spark_df.createOrReplaceTempView("osm_shapes")

# Optionally, save as a permanent table
spark_df.write.mode("overwrite").saveAsTable("osm_shapes")

In [0]:
%sql
select * from osm_shapes limit 40

In [0]:
%sql
select count(1) from osm_shapes

In [0]:
%sql
select source_file,count(1) from osm_shapes group by 1

In [0]:
%sql
SELECT
  fclass,
  COUNT(1) AS cnt,
  ROUND(100.0 * COUNT(1) / SUM(COUNT(1)) OVER (), 2) AS pct
from osm_shapes
group by 1

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select osm_id,source_file, fclass,name,count(1) over (partition by osm_id) cnt
from osm_shapes

Eliminando Duplicados y seleccionando unicamente las capas:
- gis_osm_pois_a_free_1
- gis_osm_pofw_a_free_1
- gis_osm_buildings_a_free_1
- gis_osm_landuse_a_free_1
- gis_osm_traffic_a_free_1

Asi mismo aplicando el siguiente orden de prioridad como criterio para seleccionar el registro con el que nos quedaremos luego de quitar duplicados:
1. gis_osm_pois_a_free_1
2. gis_osm_pofw_a_free_1
3. gis_osm_buildings_a_free_1
4. gis_osm_landuse_a_free_1
5. gis_osm_traffic_a_free_1


In [0]:
%sql
CREATE TABLE osm_shapes_deduped AS
with raw as (
select osm_id,
       source_file,
       fclass clase,
       coalesce(name,fclass) nombre,
       case when source_file='sv_gis_osm_pois_a_free_1' then 1
            when source_file='sv_gis_osm_pofw_a_free_1' then 2
            when source_file='sv_gis_osm_buildings_a_free_1' then 3
            when source_file='sv_gis_osm_landuse_a_free_1' then 4
            when source_file='sv_gis_osm_traffic_a_free_1' then 5
        else 6 end priority,
      geometry_wkt
from osm_shapes
where source_file in ('sv_gis_osm_pois_a_free_1','sv_gis_osm_pofw_a_free_1','sv_gis_osm_buildings_a_free_1','sv_gis_osm_landuse_a_free_1','sv_gis_osm_traffic_a_free_1')
)
, row_numbered as (
select *, row_number() over (partition by osm_id order by priority) rn
from raw
)
select * except (rn)
from row_numbered 
where rn=1


In [0]:
%sql
select count(1) from osm_shapes_deduped

Verificando que no hay duplicados

In [0]:
%sql
select osm_id,source_file, clase,nombre,count(1) over (partition by osm_id) cnt
from osm_shapes_deduped
order by 5 desc

Verificacion que los poligonos de OSM si esten dentro de El Salvador

In [0]:
df_shapes = spark.table("osm_shapes_deduped").toPandas()

df_shapes["geometry"] = df_shapes["geometry_wkt"].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df_shapes, geometry="geometry", crs="EPSG:4326")

#Poligono de El Salvador
elsalvador_bbox = Polygon([
    (-90.13, 13.14),
    (-90.13, 14.45),
    (-87.69, 14.45),
    (-87.69, 13.14),
    (-90.13, 13.14)
])
gdf_elsalvador = gpd.GeoDataFrame(index=[0], geometry=[elsalvador_bbox], crs="EPSG:4326")
filtered = gdf[gdf.within(gdf_elsalvador.loc[0, "geometry"])]

In [0]:
gdf.count()

In [0]:
filtered = gdf[gdf.within(gdf_elsalvador.loc[0, "geometry"])]

In [0]:
filtered.count()

Persistiendo en tabla

In [0]:
# Convert GeoDataFrame to Pandas DataFrame (geometry as WKT)
filtered['geometry_wkt'] = filtered['geometry'].apply(lambda geom: geom.wkt if geom is not None else None)
pdf_filtered = filtered.drop(columns='geometry')

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(pdf_filtered)

# Register as a temporary view or save as a table
spark_df.createOrReplaceTempView("osm_shapes_filtered")

# Optionally, save as a permanent table
spark_df.write.mode("overwrite").saveAsTable("osm_shapes_filtered")

In [0]:
%sql
select count(1) from osm_shapes_filtered